In [32]:
import pandas as pd
import numpy as np
import re


In [2]:
player_stats = pd.read_csv("merged_df.csv")

In [3]:
play_data = pd.read_csv("./data2023/plays.csv")

In [4]:
player_stats.head()

,Unnamed: 0,Acceleration,Age,Agility,Archetype,Awareness,BC Vision,Block Shedding,Break Sack,Break Tackle,...,Zone Coverage,birthDate,collegeName,displayName,height,nflId,officialPosition,updatedName,updatedName2,weight
0,0,66,44,70,QB_FieldGeneral,99,51,33,61,31,...,19,1977-08-03,Michigan,Tom Brady,6-4,25511,QB,Tom Brady,Tom Brady,225
1,1,77,39,67,QB_FieldGeneral,85,68,45,83,64,...,13,1982-03-02,"Miami, O.",Ben Roethlisberger,6-5,28963,QB,Ben Roethlisberger,Ben Roethlisberger,240
2,2,84,37,81,QB_Improviser,96,78,19,86,65,...,19,1983-12-02,California,Aaron Rodgers,6-2,29851,QB,Aaron Rodgers,Aaron Rodgers,225
3,3,79,38,71,QB_FieldGeneral,82,73,12,82,66,...,12,1982-11-24,Harvard,Ryan Fitzpatrick,6-2,30078,QB,Ryan Fitzpatrick,Ryan Fitzpatrick,228
4,4,80,37,72,TE_Blocking,76,75,45,11,54,...,13,1984-05-19,UCLA,Marcedes Lewis,6-6,30842,TE,Marcedes Lewis,Marcedes Lewis,267


In [5]:
play_data.head()

,gameId,playId,playDescription,quarter,down,yardsToGo,possessionTeam,defensiveTeam,yardlineSide,yardlineNumber,...,foulNFLId3,absoluteYardlineNumber,offenseFormation,personnelO,defendersInBox,personnelD,dropBackType,pff_playAction,pff_passCoverage,pff_passCoverageType
0,2021090900,97,(13:33) (Shotgun) T.Brady pass incomplete deep...,1,3,2,TB,DAL,TB,33,...,NaN,43.0,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"4 DL, 2 LB, 5 DB",TRADITIONAL,0,Cover-1,Man
1,2021090900,137,(13:18) (Shotgun) D.Prescott pass deep left to...,1,1,10,DAL,TB,DAL,2,...,NaN,108.0,EMPTY,"1 RB, 2 TE, 2 WR",6.0,"4 DL, 4 LB, 3 DB",TRADITIONAL,0,Cover-3,Zone
2,2021090900,187,(12:23) (Shotgun) D.Prescott pass short middle...,1,2,6,DAL,TB,DAL,34,...,NaN,76.0,SHOTGUN,"0 RB, 2 TE, 3 WR",6.0,"3 DL, 3 LB, 5 DB",TRADITIONAL,0,Cover-3,Zone
3,2021090900,282,(9:56) D.Prescott pass incomplete deep left to...,1,1,10,DAL,TB,TB,39,...,NaN,49.0,SINGLEBACK,"1 RB, 2 TE, 2 WR",6.0,"4 DL, 3 LB, 4 DB",TRADITIONAL,1,Cover-3,Zone
4,2021090900,349,(9:46) (Shotgun) D.Prescott pass incomplete sh...,1,3,15,DAL,TB,TB,44,...,NaN,54.0,SHOTGUN,"1 RB, 1 TE, 3 WR",7.0,"3 DL, 4 LB, 4 DB",TRADITIONAL,0,Cover-3,Zone


In [6]:
# Remove time from play description
playDescText = play_data.playDescription.apply(lambda x: " ".join(str(x).split()[1:]))
# Remove formation from play description
playDescText = playDescText.apply(lambda x: "".join(x.split(")")[1:]) if str(x).startswith("(") else x)

In [7]:
print(playDescText)

0         T.Brady pass incomplete deep right to C.Godwin.
1        D.Prescott pass deep left to A.Cooper pushed ...
2        D.Prescott pass short middle to D.Schultz to ...
3         D.Prescott pass incomplete deep left to C.Lamb.
4        D.Prescott pass incomplete short left to C.La...
                              ...                        
8552     P.Mahomes sacked at NYG 16 for -8 yards (K.Cr...
8553     D.Jones pass short right to E.Engram pushed o...
8554      D.Jones sacked at NYG 20 for -8 yards (C.Jones.
8555     D.Jones pass incomplete short right to E.Engram.
8556     D.Jones sacked at NYG 14 for -6 yards (F.Clar...
Name: playDescription, Length: 8557, dtype: object


In [26]:
# Create binary variable for whether ball was passed or sacked
passed = playDescText.map(lambda x: 1 if "pass" in str(x).lower() else 0)
sacked = playDescText.map(lambda x: 1 if "sacked" in str(x).lower() else 0)
fumbled = playDescText.map(lambda x: 1 if "fumble" in str(x).lower() else 0)
scrambled = playDescText.map(lambda x: 1 if "scramble" in str(x).lower() else 0)
print(passed.sum(), "passed")
print(sacked.sum(), "sacked")
print(fumbled.sum(), "fumbled")
print(scrambled.sum(), "scrambled")
print(passed.sum() + sacked.sum() + fumbled.sum() + scrambled.sum(), " total")

7575 passed
543 sacked
151 fumbled
449 scrambled
8718  total


In [40]:
others = playDescText.loc[fumbled==1]
others.head()

32      D.Prescott to TB 10 for -5 yards. FUMBLES, an...
142     J.Hurts to ATL 25 for -5 yards. FUMBLES, and ...
182     B.Roethlisberger sacked at PIT 20 for -6 yard...
194     J.Allen sacked at PIT 45 for -7 yards (T.Watt...
234     J.Allen pass short right to D.Singletary to P...
Name: playDescription, dtype: object

In [37]:
passers = playDescText.map(lambda x: str(x).lower().split(" ")[str(x).lower().split(" ").index("pass")-1] if "pass" in str(x).lower().split(" ") else np.nan)
receivers = playDescText.map(lambda x: str(x).lower().split(" ")[str(x).lower().split(" ").index("to")+1] if ("pass" in str(x).lower().split(" ") and "to" in str(x).lower().split(" ")) else np.nan)
scramblers = playDescText.map(lambda x: str(x).lower().split(" ")[str(x).lower().split(" ").index("scrambles")-1] if ("scramble" in str(x).lower()) else np.nan)
sackees = playDescText.map(lambda x: str(x).lower().split(" ")[str(x).lower().split(" ").index("sacked")-1] if ("sacked" in str(x).lower().split(" ")) else np.nan)
sackers = playDescText.map(lambda x: re.findall("\[(.*?)\]", str(x))[0] if (re.findall("\[(.*?)\]", str(x))) and ("sacked" in str(x).lower()) else np.nan)

In [29]:
passers.describe()

count       7567
unique        59
top       j.goff
freq         296
Name: playDescription, dtype: object

In [30]:
receivers.describe()

count        7253
unique        742
top       a.brown
freq           68
Name: playDescription, dtype: object

In [31]:
scramblers.describe()

count         449
unique         50
top       j.hurts
freq           33
Name: playDescription, dtype: object

In [38]:
sackers.describe()

count         54
unique        43
top       T.Watt
freq           3
Name: playDescription, dtype: object